<a href="https://colab.research.google.com/github/ruitenbeek/thesis/blob/main/2step_bertje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed May 26 10:06:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 26.4MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 3.3MB 33.7MB/s 
     |████████████████████████████████| 1.8MB 41.1MB/s 
     |████████████████████████████████| 225kB 58.2MB/s 
     |████████████████████████████████| 122kB 53.9MB/s 
     |████████████████████████████████| 8.2MB 54.6MB/s 
     |████████████████████████████████| 2.3MB 48.6MB/s 
     |████████████████████████████████| 1.2MB 51.8MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 174kB 58.1MB/s 
     |████████████████████████████████| 133kB 52.6MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 245kB 22.8MB/s 
     |████████████████████████████████| 112kB 41.4MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 4.2MB 44.6MB

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/thesis/code')
!pwd

/content/gdrive/My Drive/thesis/code


#Import

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from csv import DictReader
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

#Read Files

In [ ]:
def read_train_file(file):
    data1 = list()
    data2 = list()
    abu_count = 0
    off_count = 0
    not_count = 0
    with open(file, 'r') as f:
        reader = DictReader(f, delimiter='\t')
        for row in reader:
            if (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and (row['explicitness'] == 'IMPLICIT' or row['explicitness'] == 'EXPLICIT'):
                data1.append([row['text'], 'OFF'])
                data2.append([row['text'], 'OFF'])
                off_count += 1
            elif row['abusive'] == 'IMPLICIT' or row['abusive'] == 'EXPLICIT':
                data1.append([row['text'], 'OFF'])
                data2.append([row['text'], 'ABU'])
                abu_count += 1
            elif (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and row['explicitness'] == 'NOT':
                data1.append([row['text'], 'NOT'])
                not_count += 1
    print(f'ABU: %i\nOFF: %i\nNOT: %i' % (abu_count, off_count, not_count))
    data1_df = pd.DataFrame(data1)
    data1_df.columns = ['text', 'label']
    data2_df = pd.DataFrame(data2)
    data2_df.columns = ['text', 'label']
    return data1_df, data2_df


In [ ]:
def read_test_file(file):
    data1 = list()
    data2 = list()
    abu_count = 0
    off_count = 0
    not_count = 0
    with open(file, 'r') as f:
        reader = DictReader(f, delimiter='\t')
        for row in reader:
            if (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and (row['explicitness'] == 'IMPLICIT' or row['explicitness'] == 'EXPLICIT'):
                data1.append([row['text'], 'OFF'])
                data2.append([row['text'], 'OFF'])
                off_count += 1
            elif row['abusive'] == 'IMPLICIT' or row['abusive'] == 'EXPLICIT':
                data1.append([row['text'], 'OFF'])
                data2.append([row['text'], 'ABU'])
                abu_count += 1
            elif (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and row['explicitness'] == 'NOT':
                data1.append([row['text'], 'NOT'])
                data2.append([row['text'], 'NOT'])
                not_count += 1
    print(f'ABU: %i\nOFF: %i\nNOT: %i' % (abu_count, off_count, not_count))
    data1_df = pd.DataFrame(data1)
    data1_df.columns = ['text', 'label']
    data2_df = pd.DataFrame(data2)
    data2_df.columns = ['text', 'label']
    return data1_df, data2_df

#Model

In [ ]:
def bertje1(train_df):
    model_args = ClassificationArgs(num_train_epochs=4, train_batch_size=64, learning_rate=5e-5, overwrite_output_dir=True, silent=True)
    model = ClassificationModel('bert', 'GroNLP/bert-base-dutch-cased', args=model_args, use_cuda=True, num_labels=3)
    model.train_model(train_df)
    return model

In [ ]:
def bertje2(train_df):
    model_args = ClassificationArgs(num_train_epochs=4, train_batch_size=64, learning_rate=5e-5, overwrite_output_dir=True, silent=True)
    model = ClassificationModel('bert', 'GroNLP/bert-base-dutch-cased', args=model_args, use_cuda=True, num_labels=3)
    model.train_model(train_df)
    return model

#Evaluation

In [ ]:
def evaluation(model1, model2, test_df1, test_df2):
    test_X1 = test_df1.text.tolist()
    test_y1 = test_df1.label.tolist()
    test_y2 = test_df2.label.tolist()
    
    pred_y1, raw_outputs = model1.predict(test_X1)
    print(classification_report(test_y1, pred_y1, target_names=['NOT', 'OFF'], digits=2))
    
    for i in range(len(pred_y1)):
        if pred_y1[i] == 2:
            # If predicted label is offensive, predict new label using model 2 and change the offensive label in pred_y1 to new label
            [pred_y1[i]], raw_outputs = model2.predict([test_X1[i]])
    print(classification_report(test_y2, pred_y1, target_names=['ABU', 'NOT', 'OFF'], digits=2))

# Main

In [ ]:
print('###TRAIN Split###')
train_data1, train_data2 = read_train_file('train_final_pp.csv')
print('\n###DEV Split###')
dev_data1, dev_data2 = read_test_file('dev_final_pp.csv')
print('\n###TEST Split###')
test_data1, test_data2 = read_test_file('test_final_pp.csv')
le = LabelEncoder()
le.fit(['NOT', 'OFF', 'ABU'])
train_data1['label'] = le.transform(train_data1['label'])
train_data2['label'] = le.transform(train_data2['label'])
dev_data1['label'] = le.transform(dev_data1['label'])
dev_data2['label'] = le.transform(dev_data2['label'])
test_data1['label'] = le.transform(test_data1['label'])
test_data2['label'] = le.transform(test_data2['label'])

###TRAIN Split###
ABU: 1143
OFF: 1445
NOT: 5176

###DEV Split###
ABU: 110
OFF: 76
NOT: 361

###TEST Split###
ABU: 637
OFF: 399
NOT: 2072


In [ ]:
for i in range(5):
    print(f'\n###RUN %i###' % i)
    model1 = bertje1(train_data1)
    model2 = bertje2(train_data2)
    #print('\n####DEV RESULTS####')
    #evaluation(model1, model2, dev_data1, dev_data2)
    print('\n####TEST RESULTS####')
    evaluation(model1, model2, test_data1, test_data2)


###RUN 0###


Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/437M [00:00<?, ?B/s]

Some weights of the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

...


####TEST RESULTS####
              precision    recall  f1-score   support

         NOT       0.90      0.91      0.90      2072
         OFF       0.81      0.79      0.80      1036

    accuracy                           0.87      3108
   macro avg       0.86      0.85      0.85      3108
weighted avg       0.87      0.87      0.87      3108

              precision    recall  f1-score   support

         ABU       0.68      0.47      0.56       637
         NOT       0.90      0.91      0.90      2072
         OFF       0.38      0.55      0.45       399

    accuracy                           0.77      3108
   macro avg       0.65      0.64      0.64      3108
weighted avg       0.79      0.77      0.77      3108


###RUN 3###


Some weights of the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base


####TEST RESULTS####
              precision    recall  f1-score   support

         NOT       0.91      0.89      0.90      2072
         OFF       0.78      0.82      0.80      1036

    accuracy                           0.86      3108
   macro avg       0.85      0.85      0.85      3108
weighted avg       0.87      0.86      0.87      3108

              precision    recall  f1-score   support

         ABU       0.64      0.45      0.53       637
         NOT       0.91      0.89      0.90      2072
         OFF       0.34      0.54      0.41       399

    accuracy                           0.75      3108
   macro avg       0.63      0.62      0.61      3108
weighted avg       0.78      0.75      0.76      3108


###RUN 4###


Some weights of the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base


####TEST RESULTS####
              precision    recall  f1-score   support

         NOT       0.89      0.92      0.90      2072
         OFF       0.82      0.76      0.79      1036

    accuracy                           0.87      3108
   macro avg       0.85      0.84      0.85      3108
weighted avg       0.86      0.87      0.86      3108

              precision    recall  f1-score   support

         ABU       0.67      0.45      0.54       637
         NOT       0.89      0.92      0.90      2072
         OFF       0.36      0.49      0.41       399

    accuracy                           0.77      3108
   macro avg       0.64      0.62      0.62      3108
weighted avg       0.77      0.77      0.76      3108

